In [1]:
import networkx as nx
import numpy as np
import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
# with open("hepth/Cit-HepTh.txt") as f:
#     edges = [line.split() for line in f]
    
# for e in edges:
#     e[0] = '0' * (7 - len(e[0])) + e[0]
#     e[1] = '0' * (7 - len(e[1])) + e[1]
    
# with open("hepth/Cit-HepTh-dates.txt") as f:
#     dates = [(line.split()[0], parser.parse(line.split()[1])) for line in f]
# #     dates = [(int(line.split()[0]), parser.parse(line.split()[1])) for line in f]
    
# dates = [d if d[0][:2] != '11' else (d[0][2:], d[1]) for d in dates]
    
# print(len(edges))
# print(len(dates))
    
# edges = sorted(edges)
# dates.sort(key=lambda x: x[1])

In [3]:
with open("hepth/hep-th-citations") as f:
    edges = [(int(line.split()[0]), int(line.split()[1])) for line in f]
    
with open("hepth/hep-th-slacdates") as f:
#     dates = [(line.split()[0], parser.parse(line.split()[1])) for line in f]
    dates = [(int(line.split()[0]), parser.parse(line.split()[1])) for line in f]
    
# dates = [(int(d[0]), d[1]) if d[0][:2] != '11' else (int(d[0][2:]), d[1]) for d in dates]

print(len(edges))
print(len(dates))
    
edges = sorted(edges)
# dates = sorted(dates)
dates.sort(key=lambda x: x[1])

352807
29555


In [4]:
# cites = [x[0] for x in edges]

# dates = [d for d in dates if d[0] in cites]
# edges = [x for x in edges if x[1] in cites]

print(len(dates))
dates[:5]

29555


[(9705032, datetime.datetime(1976, 1, 1, 0, 0)),
 (1113, datetime.datetime(1982, 1, 1, 0, 0)),
 (9911050, datetime.datetime(1984, 10, 1, 0, 0)),
 (9910093, datetime.datetime(1985, 7, 1, 0, 0)),
 (9512041, datetime.datetime(1986, 10, 28, 0, 0))]

In [5]:
print(len(edges))
edges[:5]

352807


[(1001, 9304045),
 (1001, 9308122),
 (1001, 9309097),
 (1001, 9311042),
 (1001, 9401139)]

In [19]:
times = dict(dates)

# valids = [x[0] for x in dates]
    
# timed_edges = [(x, max(times[x[0]], times[x[1]])) for x in edges if x[0] in valids and x[1] in valids]
# timed_edges = [(x, max(times[x[0]], times[x[1]])) for x in edges]
timed_edges = [(x, times[x[0]]) for x in edges]
timed_edges.sort(key=lambda x: x[1])
timed_edges[:5]

[((9910093, 9810028), datetime.datetime(1985, 7, 1, 0, 0)),
 ((9910093, 9906064), datetime.datetime(1985, 7, 1, 0, 0)),
 ((9910093, 9908186), datetime.datetime(1985, 7, 1, 0, 0)),
 ((9910093, 9910093), datetime.datetime(1985, 7, 1, 0, 0)),
 ((9201015, 9207016), datetime.datetime(1992, 1, 6, 0, 0))]

In [12]:
cut_pts = pd.date_range('1993-1-1', '1994-1-1', freq='MS')
cut_pts = list(cut_pts)[:10]

print(len(cut_pts))
cut_pts[:5]

10


[Timestamp('1993-01-01 00:00:00', freq='MS'),
 Timestamp('1993-02-01 00:00:00', freq='MS'),
 Timestamp('1993-03-01 00:00:00', freq='MS'),
 Timestamp('1993-04-01 00:00:00', freq='MS'),
 Timestamp('1993-05-01 00:00:00', freq='MS')]

In [31]:
graphs = []

for cut in tqdm_notebook(cut_pts):
#     cut = all_dates[cut - 1]
    nodes = [x[0] for x in dates if x[1] < cut]
#     adds = [x[0] for x in timed_edges if x[1] < cut]
    adds = [x for x in edges if x[0] in nodes and x[1] in nodes]
    g = nx.DiGraph()
#     g.add_nodes_from(nodes)
    g.add_edges_from(adds)
    
#     g = max(nx.connected_component_subgraphs(nx.Graph(g)), key=len)
#     g = max(nx.strongly_connected_component_subgraphs(g), key=len)
    
    graphs.append(g)
    print(cut)
    print(graphs[-1].number_of_nodes())
    print(graphs[-1].number_of_edges())
    print("============")


1993-01-01 00:00:00
491
585
1993-02-01 00:00:00
567
717
1993-03-01 00:00:00
666
885
1993-04-01 00:00:00
812
1139
1993-05-01 00:00:00
948
1424
1993-06-01 00:00:00
1074
1734
1993-07-01 00:00:00
1224
2103
1993-08-01 00:00:00
1365
2459
1993-09-01 00:00:00
1510
2863
1993-10-01 00:00:00
1638
3222



In [29]:
node_dict = {}
cur = 0

for g in graphs:
    old_cur = cur
    for node in g.nodes:
        if not node in node_dict:
            node_dict[node] = cur
            cur += 1
            
    print("new nodes: ", cur - old_cur)
            
for i in range(len(graphs)):
    graphs[i] = nx.relabel_nodes(graphs[i], node_dict)

new nodes:  505
new nodes:  78
new nodes:  102
new nodes:  146
new nodes:  137
new nodes:  141
new nodes:  146
new nodes:  140
new nodes:  141
new nodes:  127


In [15]:
for i in range(len(graphs)):
    nx.write_graphml(graphs[i], 'hepth/graphs/' + str(i) + '.graphml')

In [32]:
print(nx.algorithms.cluster.average_clustering(graphs[0]))
print(nx.algorithms.cluster.average_clustering(graphs[-1]))

0.06277883044435466
0.08351501067683696
